In [1]:
cd ..

/Users/muhammadfarooq/Codebase/thesis-experiment/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

# Damage Dataset

In [3]:
def read_damage_min_max(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_mean']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_min_max': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [4]:
def read_damage_mean(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_min_max']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_mean': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [5]:
#damage
damage_sixteen_url = "damage/damage_sixteen.csv"
damage_thirtytwo_url = "damage/damage_thirtytwo.csv"

df_damage_sixteen_min_max = read_damage_min_max(damage_sixteen_url)
df_damage_thirtytwo_min_max = read_damage_min_max(damage_thirtytwo_url)

df_damage_sixteen_mean = read_damage_mean(damage_sixteen_url)
df_damage_thirtytwo_mean = read_damage_mean(damage_thirtytwo_url)

In [6]:
#df_damage_sixteen_min_max

In [7]:
#df_damage_thirtytwo_min_max

In [8]:
#df_damage_sixteen_mean

In [9]:
#df_damage_thirtytwo_mean

In [10]:
#STOP

In [11]:
df = df_damage_sixteen_min_max

In [12]:
#df

In [13]:
# set up the file paths for your image data and labels
img_size = (128, 128)

data_dir = 'damage/images'

# convert the DataFrame to a dictionary with specific column names as key and value
label_dict = {k: v for k, v in zip(df['image'], df['Damage'])}

# create empty lists to store the image data and labels
images = []
labels = []

# loop through each image file in the directory
for filename in os.listdir(data_dir):
    # load the image file and convert it to a NumPy array
    image = load_img(os.path.join(data_dir, filename), target_size=img_size)
    image = img_to_array(image)
    # preprocess the image by scaling the pixel values to be between 0 and 1
    image = image.astype('float32') / 255.0
    # add the preprocessed image and its label to the lists
    images.append(image)
    labels.append(label_dict[filename])

# convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# print the shape of the data to verify that it's been loaded correctly
print('Image data shape:', images.shape) #(148, 64, 64, 3)
print('Label data shape:', labels.shape) #(148,)

Image data shape: (148, 128, 128, 3)
Label data shape: (148,)


In [14]:
#STOP

# Config

In [15]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [16]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [17]:
units = [32]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

In [18]:
#STOP

# Model

In [19]:
class Baseline():
    def __init__(self, _input_dim, units, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # convolutional layer
        model.add(Conv2D(units, kernel_size=kernel_size, activation=activation, input_shape=_input_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
                  
        # flatten output of conv
        model.add(Flatten())
                  
        # hidden layer
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout))
                  
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [20]:
X = images
y = labels

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [21]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [22]:
input_dim = X_train.shape[1:]

In [23]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
                            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
                            
                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 32 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 1s - loss: 1.9462 - accuracy: 0.4886 - val_loss: 1.1957 - val_accuracy: 0.8444 - 1s/epoch - 473ms/step
Epoch 2/50
3/3 - 0s - loss: 1.9829 - accuracy: 0.7841 - val_loss: 2.9646 - val_accuracy: 0.7556 - 433ms/epoch - 144ms/step
Epoch 3/50
3/3 - 0s - loss: 1.3217 - accuracy: 0.8295 - val_loss: 1.6535 - val_accuracy: 0.7556 - 427ms/epoch - 142ms/step
Epoch 4/50
3/3 - 0s - loss: 1.1606 - accuracy: 0.8523 - val_loss: 0.7172 - val_accuracy: 0.8667 - 476ms/epoch - 159ms/step
Epoch 5/50
3/3 - 0s - loss: 0.7211 - accuracy: 0.8523 - val_loss: 0.4719 - val_accuracy: 0.8667 - 489ms/epoch - 163ms/step
Epoch 6/50
3/3 - 1s - loss: 0.5580 - accuracy: 0.8636 - val_loss: 0.3423 - val_accuracy: 0.8667 - 503ms/epoch - 168ms/step
Epoch 7/50
3/3 - 0s - loss: 0.4112 - accuracy: 0.8636 - val_loss: 0.3628 - val_accuracy: 0.8667 - 480ms/epoch - 160ms/step
Epoch 8/50
3/3 - 0s - loss: 0.3997 - accuracy: 0.

Epoch 20/50
3/3 - 0s - loss: 0.0661 - accuracy: 0.9888 - val_loss: 0.3071 - val_accuracy: 0.9318 - 453ms/epoch - 151ms/step
Epoch 21/50
3/3 - 0s - loss: 0.0386 - accuracy: 1.0000 - val_loss: 0.3273 - val_accuracy: 0.8864 - 435ms/epoch - 145ms/step
Epoch 22/50
3/3 - 0s - loss: 0.0331 - accuracy: 1.0000 - val_loss: 0.3894 - val_accuracy: 0.8864 - 440ms/epoch - 147ms/step

Units: 32 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.2
Epoch 1/50
3/3 - 1s - loss: 1.2839 - accuracy: 0.4659 - val_loss: 1.3497 - val_accuracy: 0.3778 - 1s/epoch - 393ms/step
Epoch 2/50
3/3 - 0s - loss: 0.7089 - accuracy: 0.7273 - val_loss: 0.5756 - val_accuracy: 0.8667 - 443ms/epoch - 148ms/step
Epoch 3/50
3/3 - 0s - loss: 0.7133 - accuracy: 0.8068 - val_loss: 0.4988 - val_accuracy: 0.8444 - 448ms/epoch - 149ms/step
Epoch 4/50
3/3 - 0s - loss: 0.4756 - accuracy: 0.8636 - val_loss: 0.4830 - val_accuracy: 0.7556 - 442ms/epoch - 147ms/step
Epoch 5/50
3/3 - 0s - loss: 0.4540 - accuracy:

Epoch 9/50
3/3 - 0s - loss: 0.4530 - accuracy: 0.8295 - val_loss: 0.4560 - val_accuracy: 0.7556 - 437ms/epoch - 146ms/step
Epoch 10/50
3/3 - 0s - loss: 0.3747 - accuracy: 0.8636 - val_loss: 0.4317 - val_accuracy: 0.8000 - 450ms/epoch - 150ms/step
Epoch 11/50
3/3 - 0s - loss: 0.3952 - accuracy: 0.8864 - val_loss: 0.4169 - val_accuracy: 0.8000 - 451ms/epoch - 150ms/step
Epoch 12/50
3/3 - 0s - loss: 0.3914 - accuracy: 0.8750 - val_loss: 0.4119 - val_accuracy: 0.8222 - 453ms/epoch - 151ms/step
Epoch 13/50
3/3 - 0s - loss: 0.3825 - accuracy: 0.8636 - val_loss: 0.4020 - val_accuracy: 0.8222 - 444ms/epoch - 148ms/step
Epoch 14/50
3/3 - 0s - loss: 0.3101 - accuracy: 0.8864 - val_loss: 0.4616 - val_accuracy: 0.8222 - 443ms/epoch - 148ms/step
Epoch 15/50
3/3 - 0s - loss: 0.3535 - accuracy: 0.8977 - val_loss: 0.3761 - val_accuracy: 0.8444 - 462ms/epoch - 154ms/step
Epoch 16/50
3/3 - 0s - loss: 0.3503 - accuracy: 0.8409 - val_loss: 0.3856 - val_accuracy: 0.8667 - 440ms/epoch - 147ms/step
Epoch 17/

Epoch 5/50
3/3 - 0s - loss: 0.5117 - accuracy: 0.8068 - val_loss: 0.9370 - val_accuracy: 0.4222 - 408ms/epoch - 136ms/step
Epoch 6/50
3/3 - 0s - loss: 0.5376 - accuracy: 0.7386 - val_loss: 0.4481 - val_accuracy: 0.7778 - 426ms/epoch - 142ms/step
Epoch 7/50
3/3 - 0s - loss: 0.3971 - accuracy: 0.8409 - val_loss: 0.4019 - val_accuracy: 0.8000 - 426ms/epoch - 142ms/step
Epoch 8/50
3/3 - 0s - loss: 0.3696 - accuracy: 0.8295 - val_loss: 0.9397 - val_accuracy: 0.3556 - 404ms/epoch - 135ms/step
Epoch 9/50
3/3 - 0s - loss: 0.4269 - accuracy: 0.7159 - val_loss: 0.4110 - val_accuracy: 0.8444 - 430ms/epoch - 143ms/step
Epoch 10/50
3/3 - 0s - loss: 0.4306 - accuracy: 0.8068 - val_loss: 0.5650 - val_accuracy: 0.7556 - 413ms/epoch - 138ms/step
Epoch 11/50
3/3 - 0s - loss: 0.3833 - accuracy: 0.8523 - val_loss: 0.4034 - val_accuracy: 0.8667 - 430ms/epoch - 143ms/step
Epoch 12/50
3/3 - 0s - loss: 0.4684 - accuracy: 0.8295 - val_loss: 0.3745 - val_accuracy: 0.8667 - 423ms/epoch - 141ms/step
Epoch 13/50
3

Epoch 24/50
3/3 - 0s - loss: 0.2170 - accuracy: 0.8989 - val_loss: 0.6506 - val_accuracy: 0.7727 - 450ms/epoch - 150ms/step
Epoch 25/50
3/3 - 0s - loss: 0.2247 - accuracy: 0.9438 - val_loss: 0.6933 - val_accuracy: 0.8182 - 448ms/epoch - 149ms/step
Epoch 26/50
3/3 - 0s - loss: 0.1832 - accuracy: 0.9101 - val_loss: 0.6776 - val_accuracy: 0.8182 - 449ms/epoch - 150ms/step
Epoch 27/50
3/3 - 0s - loss: 0.2247 - accuracy: 0.9213 - val_loss: 0.7035 - val_accuracy: 0.8182 - 433ms/epoch - 144ms/step
Epoch 28/50
3/3 - 0s - loss: 0.1758 - accuracy: 0.9101 - val_loss: 0.7105 - val_accuracy: 0.8182 - 443ms/epoch - 148ms/step
Epoch 29/50
3/3 - 0s - loss: 0.2069 - accuracy: 0.8989 - val_loss: 0.9345 - val_accuracy: 0.8182 - 436ms/epoch - 145ms/step
Epoch 30/50
3/3 - 0s - loss: 0.1991 - accuracy: 0.9326 - val_loss: 0.8328 - val_accuracy: 0.8182 - 442ms/epoch - 147ms/step
Epoch 31/50
3/3 - 0s - loss: 0.1798 - accuracy: 0.9326 - val_loss: 0.7266 - val_accuracy: 0.8182 - 441ms/epoch - 147ms/step
Epoch 32

...

Epoch 10/50
3/3 - 1s - loss: 0.6512 - accuracy: 0.6966 - val_loss: 0.7665 - val_accuracy: 0.5000 - 593ms/epoch - 198ms/step
Epoch 11/50
3/3 - 1s - loss: 0.6195 - accuracy: 0.6966 - val_loss: 0.7234 - val_accuracy: 0.5000 - 592ms/epoch - 197ms/step
Epoch 12/50
3/3 - 1s - loss: 0.6383 - accuracy: 0.7191 - val_loss: 0.6925 - val_accuracy: 0.5000 - 580ms/epoch - 193ms/step
Epoch 13/50
3/3 - 1s - loss: 0.6198 - accuracy: 0.6742 - val_loss: 0.7579 - val_accuracy: 0.5000 - 585ms/epoch - 195ms/step
Epoch 14/50
3/3 - 1s - loss: 0.5979 - accuracy: 0.6854 - val_loss: 0.7508 - val_accuracy: 0.5000 - 608ms/epoch - 203ms/step
Epoch 15/50
3/3 - 1s - loss: 0.6181 - accuracy: 0.6966 - val_loss: 0.6998 - val_accuracy: 0.5000 - 593ms/epoch - 198ms/step
Epoch 16/50
3/3 - 1s - loss: 0.6208 - accuracy: 0.7079 - val_loss: 0.7694 - val_accuracy: 0.5000 - 586ms/epoch - 195ms/step
Epoch 1/50
3/3 - 1s - loss: 0.9146 - accuracy: 0.4831 - val_loss: 0.5628 - val_accuracy: 0.7500 - 1s/epoch - 434ms/step
Epoch 2/50
3

Epoch 49/50
3/3 - 1s - loss: 0.5724 - accuracy: 0.7159 - val_loss: 0.5895 - val_accuracy: 0.7556 - 594ms/epoch - 198ms/step
Epoch 50/50
3/3 - 1s - loss: 0.5608 - accuracy: 0.8409 - val_loss: 0.5878 - val_accuracy: 0.7556 - 599ms/epoch - 200ms/step
Epoch 1/50
3/3 - 2s - loss: 0.6342 - accuracy: 0.7079 - val_loss: 0.7784 - val_accuracy: 0.5000 - 2s/epoch - 824ms/step
Epoch 2/50
3/3 - 1s - loss: 0.6634 - accuracy: 0.6742 - val_loss: 0.8015 - val_accuracy: 0.5000 - 591ms/epoch - 197ms/step
Epoch 3/50
3/3 - 1s - loss: 0.6113 - accuracy: 0.6966 - val_loss: 0.8084 - val_accuracy: 0.5000 - 587ms/epoch - 196ms/step
Epoch 4/50
3/3 - 1s - loss: 0.6307 - accuracy: 0.6966 - val_loss: 0.7750 - val_accuracy: 0.5000 - 599ms/epoch - 200ms/step
Epoch 5/50
3/3 - 1s - loss: 0.6165 - accuracy: 0.6966 - val_loss: 0.7377 - val_accuracy: 0.5000 - 593ms/epoch - 198ms/step
Epoch 6/50
3/3 - 1s - loss: 0.6219 - accuracy: 0.6966 - val_loss: 0.7289 - val_accuracy: 0.5000 - 588ms/epoch - 196ms/step
Epoch 7/50
3/3 - 

Epoch 6/50
3/3 - 1s - loss: 0.6800 - accuracy: 0.6023 - val_loss: 0.6453 - val_accuracy: 0.6444 - 594ms/epoch - 198ms/step
Epoch 7/50
3/3 - 1s - loss: 0.7078 - accuracy: 0.6250 - val_loss: 0.6453 - val_accuracy: 0.6444 - 571ms/epoch - 190ms/step
Epoch 8/50
3/3 - 1s - loss: 0.6754 - accuracy: 0.6250 - val_loss: 0.6454 - val_accuracy: 0.6444 - 587ms/epoch - 196ms/step
Epoch 9/50
3/3 - 1s - loss: 0.6601 - accuracy: 0.6477 - val_loss: 0.6457 - val_accuracy: 0.6444 - 595ms/epoch - 198ms/step
Epoch 10/50
3/3 - 1s - loss: 0.6311 - accuracy: 0.6705 - val_loss: 0.6422 - val_accuracy: 0.6444 - 604ms/epoch - 201ms/step
Epoch 11/50
3/3 - 1s - loss: 0.6748 - accuracy: 0.6136 - val_loss: 0.6412 - val_accuracy: 0.6444 - 593ms/epoch - 198ms/step
Epoch 12/50
3/3 - 1s - loss: 0.6555 - accuracy: 0.6250 - val_loss: 0.6406 - val_accuracy: 0.6444 - 593ms/epoch - 198ms/step
Epoch 13/50
3/3 - 1s - loss: 0.6457 - accuracy: 0.6250 - val_loss: 0.6419 - val_accuracy: 0.6444 - 580ms/epoch - 193ms/step
Epoch 14/50


Epoch 12/50
3/3 - 1s - loss: 0.6965 - accuracy: 0.5281 - val_loss: 0.5882 - val_accuracy: 0.7500 - 633ms/epoch - 211ms/step
Epoch 13/50
3/3 - 1s - loss: 0.6872 - accuracy: 0.5843 - val_loss: 0.5888 - val_accuracy: 0.7500 - 647ms/epoch - 216ms/step
Epoch 14/50
3/3 - 1s - loss: 0.6767 - accuracy: 0.5618 - val_loss: 0.6173 - val_accuracy: 0.8636 - 598ms/epoch - 199ms/step
Epoch 15/50
3/3 - 1s - loss: 0.6582 - accuracy: 0.5506 - val_loss: 0.6206 - val_accuracy: 0.8864 - 600ms/epoch - 200ms/step
Epoch 16/50
3/3 - 1s - loss: 0.6786 - accuracy: 0.5843 - val_loss: 0.6022 - val_accuracy: 0.7955 - 597ms/epoch - 199ms/step
Epoch 17/50
3/3 - 1s - loss: 0.6738 - accuracy: 0.6180 - val_loss: 0.5858 - val_accuracy: 0.7500 - 633ms/epoch - 211ms/step
Epoch 18/50
3/3 - 1s - loss: 0.6392 - accuracy: 0.6180 - val_loss: 0.5714 - val_accuracy: 0.7500 - 620ms/epoch - 207ms/step
Epoch 19/50
3/3 - 1s - loss: 0.6514 - accuracy: 0.5955 - val_loss: 0.5784 - val_accuracy: 0.7727 - 618ms/epoch - 206ms/step
Epoch 20

Epoch 28/50
3/3 - 1s - loss: 0.6632 - accuracy: 0.6250 - val_loss: 0.6392 - val_accuracy: 0.6444 - 670ms/epoch - 223ms/step
Epoch 29/50
3/3 - 1s - loss: 0.6406 - accuracy: 0.6250 - val_loss: 0.6393 - val_accuracy: 0.6444 - 597ms/epoch - 199ms/step
Epoch 30/50
3/3 - 1s - loss: 0.6353 - accuracy: 0.6250 - val_loss: 0.6382 - val_accuracy: 0.6444 - 620ms/epoch - 207ms/step
Epoch 31/50
3/3 - 1s - loss: 0.6637 - accuracy: 0.6250 - val_loss: 0.6365 - val_accuracy: 0.6444 - 604ms/epoch - 201ms/step
Epoch 32/50
3/3 - 1s - loss: 0.6342 - accuracy: 0.6364 - val_loss: 0.6347 - val_accuracy: 0.6444 - 605ms/epoch - 202ms/step
Epoch 33/50
3/3 - 1s - loss: 0.6384 - accuracy: 0.6364 - val_loss: 0.6336 - val_accuracy: 0.6444 - 607ms/epoch - 202ms/step
Epoch 34/50
3/3 - 1s - loss: 0.6439 - accuracy: 0.6250 - val_loss: 0.6320 - val_accuracy: 0.6444 - 599ms/epoch - 200ms/step
Epoch 35/50
3/3 - 1s - loss: 0.6288 - accuracy: 0.6477 - val_loss: 0.6312 - val_accuracy: 0.6444 - 591ms/epoch - 197ms/step
Epoch 36

In [24]:
print(best_acc)
print(baseline_values)

0.9318181872367859
{'units': 32, 'activation': 'tanh', 'kernel_size': 3, 'learning_rate': 0.001, 'optimizer': 'Adam', 'dropout': 0.1}


In [25]:
#STOP